# Module 07: Scheduled Tasks & Automation

**Difficulty**: ⭐⭐⭐ (Advanced)

**Estimated Time**: 75 minutes

**Prerequisites**: 
- Completed Modules 00-06
- Understanding of subprocess and file operations
- Basic knowledge of cron/scheduled tasks
- Familiarity with logging concepts

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Create** scheduled tasks using Windows Task Scheduler
2. **Implement** Python-based scheduling solutions
3. **Build** automated data collection pipelines
4. **Configure** email notifications for automation
5. **Manage** log files and rotation strategies
6. **Handle** errors robustly in unattended scripts

## Introduction: Why Task Scheduling Matters

Automated scheduling is essential for data science workflows:

### Common Use Cases

**1. Data Collection**
- Fetch API data daily at 6 AM
- Download financial reports weekly
- Scrape websites on schedule
- Sync cloud datasets hourly

**2. Model Training**
- Retrain models with fresh data
- Run hyperparameter tuning overnight
- Generate predictions on schedule
- Update feature stores regularly

**3. Reporting**
- Email daily metrics summaries
- Generate weekly performance reports
- Monitor data quality alerts
- Archive results monthly

**4. Maintenance**
- Clean up old checkpoint files
- Compress log files
- Backup important data
- Monitor disk space

### Scheduling Methods on Windows

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **Task Scheduler** | Native, reliable, survives reboots | Complex setup | Production systems |
| **schedule library** | Easy Python integration | Requires running process | Development, testing |
| **APScheduler** | Powerful, flexible | Heavy dependency | Complex workflows |
| **Cron (WSL)** | Familiar to Linux users | Requires WSL | Cross-platform scripts |

This module focuses on Task Scheduler and the `schedule` library.

In [ ]:
# Setup: Import required libraries
import subprocess
import sys
from pathlib import Path
import time
import logging
from datetime import datetime, timedelta
import json

# Install schedule library if needed
try:
    import schedule
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'schedule', '-q'])
    import schedule

print(f"schedule version: {schedule.__version__}")
print("Setup complete!")

## 1. Windows Task Scheduler Basics

Windows Task Scheduler is the native scheduling solution. We can interact with it via Python.

In [ ]:
# List all scheduled tasks (read-only, safe operation)
def list_scheduled_tasks():
    """
    List all Windows scheduled tasks.
    
    Returns:
        list: Task names and states
    """
    try:
        # Use schtasks command to query tasks
        result = subprocess.run(
            ['schtasks', '/Query', '/FO', 'CSV'],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            # Parse CSV output
            lines = result.stdout.strip().split('\n')
            tasks = []
            
            for line in lines[1:]:  # Skip header
                parts = line.split(',')
                if len(parts) >= 2:
                    task_name = parts[0].strip('"')
                    status = parts[2].strip('"') if len(parts) > 2 else 'Unknown'
                    tasks.append({'name': task_name, 'status': status})
            
            return tasks
        else:
            return []
    
    except Exception as e:
        print(f"Error listing tasks: {e}")
        return []

# Example: List all tasks
all_tasks = list_scheduled_tasks()
print(f"Found {len(all_tasks)} scheduled tasks")

# Show first 5 as example
print("\nFirst 5 tasks:")
for task in all_tasks[:5]:
    print(f"  {task['name']}: {task['status']}")

### 1.1 Creating Scheduled Tasks

We can create scheduled tasks programmatically. This is useful for deployment automation.

In [ ]:
# Create a scheduled task (demonstration - dry run mode)
def create_scheduled_task(task_name, script_path, schedule_time, dry_run=True):
    """
    Create a Windows scheduled task.
    
    Args:
        task_name: Name for the task
        script_path: Path to Python script to run
        schedule_time: Time in HH:MM format (24-hour)
        dry_run: If True, only show command without executing
    
    Returns:
        bool: Success status
    """
    script_path = Path(script_path).resolve()
    python_exe = sys.executable
    
    # Build schtasks command
    # /Create: Create new task
    # /TN: Task name
    # /TR: Task to run
    # /SC: Schedule type (DAILY, WEEKLY, MONTHLY, ONCE)
    # /ST: Start time
    command = [
        'schtasks', '/Create',
        '/TN', task_name,
        '/TR', f'"{python_exe}" "{script_path}"',
        '/SC', 'DAILY',
        '/ST', schedule_time,
        '/F'  # Force create (overwrite if exists)
    ]
    
    print(f"{'DRY RUN: Would create' if dry_run else 'Creating'} scheduled task:")
    print(f"  Task name: {task_name}")
    print(f"  Script: {script_path}")
    print(f"  Schedule: Daily at {schedule_time}")
    print(f"  Command: {' '.join(command)}")
    
    if not dry_run:
        try:
            result = subprocess.run(command, capture_output=True, text=True)
            
            if result.returncode == 0:
                print("✓ Task created successfully")
                return True
            else:
                print(f"✗ Failed: {result.stderr}")
                return False
        
        except Exception as e:
            print(f"✗ Error: {e}")
            return False
    else:
        print("\nSet dry_run=False to actually create task")
        return True

# Example: Schedule a data collection script
# create_scheduled_task(
#     task_name='DataCollection_Daily',
#     script_path='scripts/collect_data.py',
#     schedule_time='06:00',
#     dry_run=True
# )

print("create_scheduled_task() function ready!")
print("⚠ Use with caution - always test with dry_run=True first")

## 2. Python-Based Scheduling with `schedule`

The `schedule` library provides a simple Python API for scheduling. Great for development and testing.

### When to Use `schedule` vs Task Scheduler

**Use `schedule` when:**
- Developing and testing automation scripts
- Running temporary or short-term schedules
- You need fine control within Python
- Jobs need to share Python state/memory

**Use Task Scheduler when:**
- Deploying to production
- Need jobs to run even when not logged in
- Want tasks to survive system reboots
- Need enterprise-level reliability

In [ ]:
# Basic scheduling with schedule library
import schedule
import time

# Job counter for demonstration
job_counter = {'count': 0}

def sample_job():
    """Sample job that runs on schedule."""
    job_counter['count'] += 1
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{now}] Job executed! (Run #{job_counter['count']})")

# Schedule examples
print("Schedule library examples:\n")

# Every N minutes
schedule.every(10).minutes.do(sample_job)
print("✓ Scheduled: Every 10 minutes")

# Every hour at specific minute
schedule.every().hour.at(":30").do(sample_job)
print("✓ Scheduled: Every hour at :30")

# Daily at specific time
schedule.every().day.at("09:00").do(sample_job)
print("✓ Scheduled: Daily at 09:00")

# Weekday-specific
schedule.every().monday.at("08:00").do(sample_job)
print("✓ Scheduled: Every Monday at 08:00")

# Show all scheduled jobs
print(f"\nTotal jobs scheduled: {len(schedule.get_jobs())}")

# Clear all jobs (cleanup)
schedule.clear()
print("\n(All jobs cleared for demonstration)")

### 2.1 Running the Scheduler

The `schedule` library requires a running loop to check and execute jobs.

In [ ]:
# Scheduler runner with graceful shutdown
def run_scheduler(max_iterations=None, check_interval=1):
    """
    Run the scheduler loop.
    
    Args:
        max_iterations: Maximum iterations (None = infinite)
        check_interval: Seconds between checks
    """
    print("Scheduler started. Press Ctrl+C to stop.")
    print(f"Jobs scheduled: {len(schedule.get_jobs())}\n")
    
    iteration = 0
    
    try:
        while max_iterations is None or iteration < max_iterations:
            # Run pending jobs
            schedule.run_pending()
            
            # Wait before next check
            time.sleep(check_interval)
            iteration += 1
    
    except KeyboardInterrupt:
        print("\n\nScheduler stopped by user.")
    
    print(f"Total iterations: {iteration}")

# Example: Run for 10 seconds (demonstration only)
# Schedule a job every 2 seconds
schedule.every(2).seconds.do(sample_job)

print("Demo: Running scheduler for 10 seconds...")
run_scheduler(max_iterations=10, check_interval=1)

# Cleanup
schedule.clear()
print("\nDemo complete!")

## 3. Automated Data Collection Pipeline

Let's build a realistic data collection automation that fetches API data on schedule.

In [ ]:
# Data collection automation
import json
from pathlib import Path
from datetime import datetime

class DataCollector:
    """
    Automated data collection system.
    """
    
    def __init__(self, output_dir='data/collected'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.collection_count = 0
    
    def collect_data(self, source_name, data_fetcher):
        """
        Collect data from a source.
        
        Args:
            source_name: Name of data source
            data_fetcher: Function that returns data
        """
        timestamp = datetime.now()
        
        try:
            # Fetch data
            data = data_fetcher()
            
            # Save with timestamp
            filename = f"{source_name}_{timestamp.strftime('%Y%m%d_%H%M%S')}.json"
            filepath = self.output_dir / filename
            
            with open(filepath, 'w') as f:
                json.dump({
                    'source': source_name,
                    'timestamp': timestamp.isoformat(),
                    'data': data
                }, f, indent=2)
            
            self.collection_count += 1
            print(f"✓ [{timestamp.strftime('%H:%M:%S')}] Collected {source_name}")
            print(f"  Saved to: {filepath.name}")
            
            return True
        
        except Exception as e:
            print(f"✗ [{timestamp.strftime('%H:%M:%S')}] Failed to collect {source_name}: {e}")
            return False
    
    def get_stats(self):
        """Get collection statistics."""
        files = list(self.output_dir.glob('*.json'))
        return {
            'total_collections': self.collection_count,
            'total_files': len(files),
            'output_dir': str(self.output_dir)
        }

# Example data fetcher
def fetch_sample_data():
    """Simulate fetching data from API."""
    return {
        'value': 42,
        'status': 'ok',
        'items': [1, 2, 3]
    }

# Test the collector
collector = DataCollector(output_dir='data/demo_collected')
collector.collect_data('sample_api', fetch_sample_data)

print(f"\nStats: {collector.get_stats()}")

### 3.1 Scheduled Data Collection

Combine the collector with scheduling for automated data collection.

In [ ]:
# Scheduled data collection example
import schedule

# Create collector
collector = DataCollector(output_dir='data/scheduled_collected')

# Define collection job
def scheduled_collection_job():
    """Job that collects data on schedule."""
    collector.collect_data('scheduled_api', fetch_sample_data)

# Schedule collection every 5 seconds (for demo)
schedule.every(5).seconds.do(scheduled_collection_job)

print("Demo: Scheduled data collection")
print("Collecting every 5 seconds for 15 seconds...\n")

# Run for demonstration
run_scheduler(max_iterations=15, check_interval=1)

# Show results
print(f"\nFinal stats: {collector.get_stats()}")

# Cleanup
schedule.clear()

## 4. Email Notifications

Send email alerts when automation completes, fails, or detects issues.

### Email Configuration

For production use, you'll need:
- SMTP server credentials (Gmail, Outlook, corporate server)
- App-specific password (if using Gmail)
- Email addresses for recipients

**Security Best Practice**: Store credentials in environment variables or secure configuration files, never in code!

In [ ]:
# Email notification system (demonstration)
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

class EmailNotifier:
    """
    Send email notifications for automation events.
    """
    
    def __init__(self, smtp_server, smtp_port, username, password, from_email):
        """
        Args:
            smtp_server: SMTP server address (e.g., 'smtp.gmail.com')
            smtp_port: SMTP port (587 for TLS, 465 for SSL)
            username: SMTP username
            password: SMTP password (use app-specific password)
            from_email: From email address
        """
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.username = username
        self.password = password
        self.from_email = from_email
    
    def send_notification(self, to_email, subject, body, is_html=False):
        """
        Send email notification.
        
        Args:
            to_email: Recipient email address
            subject: Email subject
            body: Email body
            is_html: If True, body is HTML
        
        Returns:
            bool: Success status
        """
        try:
            # Create message
            msg = MIMEMultipart()
            msg['From'] = self.from_email
            msg['To'] = to_email
            msg['Subject'] = subject
            
            # Attach body
            msg.attach(MIMEText(body, 'html' if is_html else 'plain'))
            
            # Connect and send
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(self.username, self.password)
                server.send_message(msg)
            
            print(f"✓ Email sent to {to_email}")
            return True
        
        except Exception as e:
            print(f"✗ Failed to send email: {e}")
            return False
    
    def send_success_notification(self, to_email, task_name, stats):
        """Send success notification with statistics."""
        subject = f"✓ {task_name} Completed Successfully"
        
        body = f"""
Task: {task_name}
Status: Success
Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Statistics:
{json.dumps(stats, indent=2)}

This is an automated message.
        """
        
        return self.send_notification(to_email, subject, body)
    
    def send_failure_notification(self, to_email, task_name, error):
        """Send failure notification with error details."""
        subject = f"✗ {task_name} Failed"
        
        body = f"""
Task: {task_name}
Status: Failed
Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Error:
{error}

Please investigate.

This is an automated message.
        """
        
        return self.send_notification(to_email, subject, body)

# Example configuration (DO NOT use real credentials here)
print("EmailNotifier class ready!")
print("\n⚠ Configuration needed:")
print("  1. Set up SMTP server credentials")
print("  2. Store in environment variables")
print("  3. Use app-specific passwords")
print("\nExample usage:")
print("""
# Load from environment
import os
notifier = EmailNotifier(
    smtp_server=os.getenv('SMTP_SERVER'),
    smtp_port=int(os.getenv('SMTP_PORT', 587)),
    username=os.getenv('SMTP_USERNAME'),
    password=os.getenv('SMTP_PASSWORD'),
    from_email=os.getenv('FROM_EMAIL')
)
""")

## 5. Logging and Error Handling

Robust logging is critical for unattended automation. You need to diagnose issues without being present.

In [ ]:
# Setup comprehensive logging
import logging
from pathlib import Path
from datetime import datetime

def setup_automation_logger(log_dir='logs', log_name='automation'):
    """
    Setup logger for automation scripts.
    
    Args:
        log_dir: Directory for log files
        log_name: Base name for log files
    
    Returns:
        logging.Logger: Configured logger
    """
    # Create logs directory
    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)
    
    # Create logger
    logger = logging.getLogger(log_name)
    logger.setLevel(logging.DEBUG)
    
    # Clear existing handlers
    logger.handlers.clear()
    
    # File handler - daily log rotation
    today = datetime.now().strftime('%Y%m%d')
    log_file = log_dir / f"{log_name}_{today}.log"
    
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)
    
    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    
    # Formatter
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    
    # Add handlers
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    logger.info(f"Logger initialized. Log file: {log_file}")
    
    return logger

# Example usage
logger = setup_automation_logger(log_dir='logs/demo', log_name='demo_automation')

logger.debug("Debug message - detailed information")
logger.info("Info message - general information")
logger.warning("Warning message - something unexpected")
logger.error("Error message - something failed")

print("\n✓ Logger configured successfully")

### 5.1 Robust Error Handling Pattern

Automation scripts must handle errors gracefully and log everything.

In [ ]:
# Robust automation wrapper
import traceback

def robust_automation_wrapper(task_func, task_name, logger, max_retries=3):
    """
    Wrap automation task with error handling and retries.
    
    Args:
        task_func: Function to execute
        task_name: Name of task (for logging)
        logger: Logger instance
        max_retries: Maximum retry attempts
    
    Returns:
        tuple: (success: bool, result: any, error: str or None)
    """
    for attempt in range(1, max_retries + 1):
        try:
            logger.info(f"Starting {task_name} (attempt {attempt}/{max_retries})")
            
            # Execute task
            result = task_func()
            
            logger.info(f"✓ {task_name} completed successfully")
            return True, result, None
        
        except Exception as e:
            error_msg = str(e)
            error_trace = traceback.format_exc()
            
            logger.error(f"✗ {task_name} failed (attempt {attempt}/{max_retries})")
            logger.error(f"Error: {error_msg}")
            logger.debug(f"Traceback:\n{error_trace}")
            
            # If last attempt, give up
            if attempt == max_retries:
                logger.error(f"✗ {task_name} failed after {max_retries} attempts")
                return False, None, error_msg
            
            # Wait before retry (exponential backoff)
            wait_time = 2 ** attempt
            logger.info(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    
    return False, None, "Max retries exceeded"

# Example task that might fail
def unreliable_task():
    """Simulates a task that sometimes fails."""
    import random
    if random.random() < 0.5:
        raise Exception("Random failure for demonstration")
    return {"status": "success", "data": [1, 2, 3]}

# Test the wrapper
logger = setup_automation_logger(log_dir='logs/demo', log_name='robust_demo')
success, result, error = robust_automation_wrapper(
    unreliable_task,
    "Unreliable Task",
    logger,
    max_retries=3
)

if success:
    print(f"\n✓ Task succeeded: {result}")
else:
    print(f"\n✗ Task failed: {error}")

### 5.2 Log File Management

Logs accumulate over time. Implement rotation and cleanup strategies.

In [ ]:
# Log file management
from pathlib import Path
from datetime import datetime, timedelta
import gzip
import shutil

def manage_log_files(log_dir, max_age_days=30, compress_age_days=7):
    """
    Manage log files: compress old logs, delete very old logs.
    
    Args:
        log_dir: Directory containing log files
        max_age_days: Delete logs older than this
        compress_age_days: Compress logs older than this
    
    Returns:
        dict: Statistics of operations
    """
    log_dir = Path(log_dir)
    
    if not log_dir.exists():
        return {'error': 'Log directory does not exist'}
    
    now = datetime.now()
    stats = {
        'compressed': 0,
        'deleted': 0,
        'total_files': 0
    }
    
    # Process all .log files
    for log_file in log_dir.glob('*.log'):
        stats['total_files'] += 1
        
        # Get file age
        modified_time = datetime.fromtimestamp(log_file.stat().st_mtime)
        age_days = (now - modified_time).days
        
        # Delete if too old
        if age_days > max_age_days:
            log_file.unlink()
            stats['deleted'] += 1
            print(f"Deleted old log: {log_file.name} ({age_days} days old)")
        
        # Compress if moderately old
        elif age_days > compress_age_days:
            # Compress to .gz
            gz_file = log_file.with_suffix('.log.gz')
            
            with open(log_file, 'rb') as f_in:
                with gzip.open(gz_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            
            # Delete original
            log_file.unlink()
            stats['compressed'] += 1
            print(f"Compressed log: {log_file.name} → {gz_file.name}")
    
    return stats

# Example usage
# stats = manage_log_files('logs/demo', max_age_days=30, compress_age_days=7)
# print(f"\nLog management stats: {stats}")

print("manage_log_files() function ready!")
print("Recommended: Run this function weekly as a scheduled task")

## 6. Practice Exercises

### Exercise 1: Stock Data Collector

Create an automated stock data collection system:
1. Fetch stock prices from an API (use sample data)
2. Schedule collection every hour
3. Save data with timestamps
4. Send email if collection fails
5. Log all operations

**Hint**: Combine DataCollector, schedule, EmailNotifier, and logging

In [ ]:
# Exercise 1: Your solution here

class StockDataCollector:
    """
    Automated stock data collection system.
    """
    
    def __init__(self, output_dir, logger, notifier=None):
        # TODO: Initialize collector with logging and notifications
        pass
    
    def fetch_stock_data(self, symbol):
        # TODO: Fetch stock data (use sample data for now)
        pass
    
    def scheduled_collection(self):
        # TODO: Collect data with error handling
        pass

# Test your collector
# collector = StockDataCollector('data/stocks', logger)
# collector.scheduled_collection()

### Exercise 2: Model Training Scheduler

Create a scheduled ML model training system:
1. Check if new training data is available
2. Load data and train model
3. Save model with version timestamp
4. Generate performance report
5. Email report to team
6. Clean up old models (keep last 5)

**Hint**: Use robust_automation_wrapper for error handling

In [ ]:
# Exercise 2: Your solution here

class ModelTrainingScheduler:
    """
    Scheduled ML model training system.
    """
    
    def __init__(self, data_dir, model_dir, logger):
        # TODO: Initialize scheduler
        pass
    
    def check_for_new_data(self):
        # TODO: Check if new training data exists
        pass
    
    def train_model(self):
        # TODO: Train model with new data
        pass
    
    def cleanup_old_models(self, keep_count=5):
        # TODO: Delete old model files
        pass

# Test your scheduler
# scheduler = ModelTrainingScheduler('data/training', 'models', logger)
# scheduler.train_model()

### Exercise 3: System Health Monitor

Create a system monitoring automation:
1. Check disk space every 6 hours
2. Check process resource usage
3. Alert if disk usage > 90%
4. Alert if any process uses > 80% CPU for 5+ minutes
5. Generate daily health report
6. Archive reports weekly

**Hint**: Use psutil for system metrics, schedule for timing

In [ ]:
# Exercise 3: Your solution here

class SystemHealthMonitor:
    """
    Automated system health monitoring.
    """
    
    def __init__(self, logger, notifier=None):
        # TODO: Initialize monitor
        pass
    
    def check_disk_space(self):
        # TODO: Check disk usage
        pass
    
    def check_processes(self):
        # TODO: Check high-resource processes
        pass
    
    def generate_health_report(self):
        # TODO: Generate comprehensive report
        pass

# Test your monitor
# monitor = SystemHealthMonitor(logger)
# monitor.check_disk_space()
# monitor.check_processes()

## 7. Summary

### Key Concepts

1. **Task Scheduling Methods**
   - Windows Task Scheduler: Production-ready, survives reboots
   - Python `schedule` library: Development and testing
   - Choose based on requirements and environment

2. **Automation Patterns**
   - Data collection: Fetch, validate, save with timestamps
   - Processing: Load, transform, save results
   - Maintenance: Cleanup, archival, monitoring

3. **Error Handling**
   - Always use try-except blocks
   - Implement retry logic with exponential backoff
   - Log everything (successes and failures)
   - Send notifications for critical failures

4. **Logging Best Practices**
   - Log to both file and console
   - Use appropriate log levels (DEBUG, INFO, WARNING, ERROR)
   - Include timestamps and context
   - Implement log rotation to manage disk space

5. **Production Considerations**
   - Store credentials securely (environment variables)
   - Implement monitoring and alerting
   - Test thoroughly before deploying
   - Document scheduling and dependencies

### Real-World Applications

- **Daily Stock Data Collection**: Fetch market data every morning
- **Model Retraining**: Retrain ML models with fresh data weekly
- **Report Generation**: Generate and email reports daily
- **System Maintenance**: Clean logs, backup data, monitor health
- **Data Pipeline**: Orchestrate multi-step ETL processes

### What's Next?

In **Module 08: Windows Security & Permissions**, you'll learn:
- File permissions and ACLs
- User and group management
- Running scripts as administrator
- Credential management
- Security best practices

### Self-Assessment

Before moving on, make sure you can:
- [ ] Create Windows scheduled tasks programmatically
- [ ] Use Python `schedule` library for automation
- [ ] Implement robust error handling with retries
- [ ] Set up comprehensive logging
- [ ] Configure email notifications
- [ ] Manage log file rotation and cleanup

---

**Continue to Module 08** when ready!